In [1]:
import pandas as pd
import numpy as np
import math
from random import choices
from datetime import datetime
import statistics

In [2]:
import devanalyst.simulation.statics as S_
from devanalyst.simulation.businessObjects import WorkAssignments, ReleaseCycleContext, UserStory, Ticket
from devanalyst.simulation.simulationModels import BalancedAllocationModel, GreedyAllocationModel, ModelsConfig, \
DefaultCostModel, DistributedLagQualityModel, NoLaggardsAllocationModel


importing Jupyter notebook from c:\users\aleja\documents\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\statics.ipynb
importing Jupyter notebook from c:\users\aleja\documents\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\businessObjects.ipynb
importing Jupyter notebook from c:\users\aleja\documents\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\simulationModels.ipynb


In [3]:
import devanalyst.test_utils.test_utils as tu_
from devanalyst.test_utils.test_utils import ExpectedOutputCleaner

importing Jupyter notebook from c:\users\aleja\documents\code\chateauclaudia-labs\devanalyst\devanalyst\test_utils\test_utils.ipynb
importing Jupyter notebook from c:\users\aleja\documents\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\generateTimecards.ipynb


<h2>test_greedyAllocationLogs</h2>

In [4]:
# Implement test logic, and run it

#Test logic
def test_greedyAllocationLogs():    
    output = {}
    RELEASE_DURATION = 125
    SPRINT_DURATION = 10
    SPRINT = 1

    # Configure models
    model = GreedyAllocationModel() 
    modelsConfig = ModelsConfig([], [], model)
    modelsConfig.random.reset(271)

    teams_df, stories_df, globalRepo = tu_.initTestData(tu_.DEV_DF, tu_.PM_DF, \
                                                        RELEASE_DURATION, SPRINT_DURATION, modelsConfig)

    modelsConfig.globalRepo = globalRepo
    # Select a team
    teamId = teams_df['Scrum Team'][0].teamId
    
    modelsConfig.context = ReleaseCycleContext(teamId, SPRINT, SPRINT_DURATION)
       
    work = WorkAssignments(modelsConfig.context, globalRepo)
    work = model.allocate(work, modelsConfig)
    
    log_df = model.buildLog_df('Sprint 1 QA', modelsConfig.context)
        
    output['Logs'] = log_df
    return output

# Run the test
test_greedyAllocationLogs_ACTUAL = test_greedyAllocationLogs()

In [5]:
# Uncomment to update expected output to match the actual one

# Helper method
def create_greedyAllocationLogs_EXPECTED():
    tu_.createExpectedOutput(test_greedyAllocationLogs_ACTUAL['Logs'],    'simm.test_greedyAllocationLogs')

# Uncomment to update expected output to match the actual one, and then put the comment back
#create_greedyAllocationLogs_EXPECTED()

In [6]:
# Load expected output, update the EXPECTED and ACTUAL dictionaries, and check test is OK
list_cols = ['Initial Data - CURRENT_SPRINT', 'Final Data - CURRENT_SPRINT', 'Remaining Data - CURRENT_SPRINT',\
             'Initial Data - NEXT_SPRINT', 'Final Data - NEXT_SPRINT', 'Remaining Data - NEXT_SPRINT'
            ]

test_greedyAllocationLogs_EXPECTED = {}

test_greedyAllocationLogs_EXPECTED['Logs']      = tu_.loadExpectedOutput('simm.test_greedyAllocationLogs', list_cols)

# Rounding inaccuracies in saving and loading CSV will create an artificial mismatch between ACTUAL and EXPECTED
# So round EXPECTED and ACTUAL to 6 decimal places for sensitive fields (any float)
ExpectedOutputCleaner.cleanRoundingNoise(['Initial Mean - CURRENT_SPRINT', 'Final Mean - CURRENT_SPRINT', \
                                          'Remaining Mean - CURRENT_SPRINT', 'Initial Distance - CURRENT_SPRINT', \
                                          'Final Distance - CURRENT_SPRINT', 'Remaining Distance - CURRENT_SPRINT',\
                                          'Initial Mean - NEXT_SPRINT', 'Final Mean - NEXT_SPRINT', \
                                          'Remaining Mean - NEXT_SPRINT', 'Initial Distance - NEXT_SPRINT', \
                                          'Final Distance - NEXT_SPRINT', 'Remaining Distance - NEXT_SPRINT'],
                                        ['Logs'],
                                        test_greedyAllocationLogs_EXPECTED,
                                        test_greedyAllocationLogs_ACTUAL)

tu_.EXPECTED['simm.test_greedyAllocationLogs']      = test_greedyAllocationLogs_EXPECTED['Logs']

tu_.ACTUAL['simm.test_greedyAllocationLogs']        = test_greedyAllocationLogs_ACTUAL['Logs']

tu_.testOK('simm.test_greedyAllocationLogs')

True

In [ ]:
test_greedyAllocationLogs_ACTUAL['Logs'][:5]

In [ ]:
test_greedyAllocationLogs_EXPECTED['Logs'][:5]

In [ ]:
# Uncomment to interactively visualize the logs, and then comment again once interactive analysis is done. Commenting these
# lines after interactive analysis is completed is required as test harness can't load these visualiations
# libraries so leaving this uncommented will crash the entire test harness.
# NOTE: MAY NEED TO RUN TWICE, as there seems to be a bug in Jupyter Notebook so on the first run there is no output
#import devanalyst.simulation.visualizations.simm_visuals as simm_visuals
#simm_visuals.renderLog(test_greedyAllocationLogs_ACTUAL['Logs'],'b')

In [ ]:
# Uncomment to interactively visualize the logs, and then comment again once interactive analysis is done. Commenting these
# lines after interactive analysis is completed is required as test harness can't load these visualiations
# libraries so leaving this uncommented will crash the entire test harness.
# NOTE: MAY NEED TO RUN TWICE, as there seems to be a bug in Jupyter Notebook so on the first run there is no output
#import devanalyst.simulation.visualizations.simm_visuals as simm_visuals
#simm_visuals.renderLog(test_greedyAllocationLogs_EXPECTED['Logs'],'g')

<h1>test_greedyAllocation</h1>

In [15]:
# Implement test logic, and run it

#Test logic
def test_greedyAllocation():    
    output = {}
    RELEASE_DURATION = 125
    SPRINT_DURATION = 10

    # Configure models
    model = GreedyAllocationModel() 
    modelsConfig = ModelsConfig([], [], model)
    modelsConfig.random.reset(271)

    teams_df, stories_df, globalRepo = tu_.initTestData(tu_.DEV_DF, tu_.PM_DF, \
                                                        RELEASE_DURATION, SPRINT_DURATION, modelsConfig)

    modelsConfig.globalRepo = globalRepo
    # Select a team
    teamId = teams_df['Scrum Team'][0].teamId
    
    # Choose what to work on at the start of a sprint.
    SPRINT_DURATION = 10
    SPRINT = 1
    modelsConfig.context = ReleaseCycleContext(teamId, SPRINT, SPRINT_DURATION)
   
    work = WorkAssignments(modelsConfig.context, globalRepo)

    work = model.allocate(work, modelsConfig)
    
    committed_df = work.committedTime(SPRINT_DURATION)
    tasks_df = work.committedTasks()
        
    output['Committed'] = committed_df
    output['Tasks'] = tasks_df
    return output

# Run the test
test_greedyAllocation_ACTUAL = test_greedyAllocation()

In [16]:
# Uncomment to update expected output to match the actual one

# Helper method
def create_greedyAllocation_EXPECTED():
    tu_.createExpectedOutput(test_greedyAllocation_ACTUAL['Committed'],    'simm.test_greedyAllocation.Committed')
    tu_.createExpectedOutput(test_greedyAllocation_ACTUAL['Tasks'],        'simm.test_greedyAllocation.Tasks')

# Uncomment to update expected output to match the actual one, and then put the comment back
#create_greedyAllocation_EXPECTED()

In [17]:
# Load expected output, update the EXPECTED and ACTUAL dictionaries, and check test is OK
test_greedyAllocation_EXPECTED = {}

test_greedyAllocation_EXPECTED['Committed']      = tu_.loadExpectedOutput('simm.test_greedyAllocation.Committed')
test_greedyAllocation_EXPECTED['Tasks']          = tu_.loadExpectedOutput('simm.test_greedyAllocation.Tasks')

# Rounding inaccuracies in saving and loading CSV will create an artificial mismatch between ACTUAL and EXPECTED
# So round EXPECTED and ACTUAL to 6 decimal places for sensitive fields (any float)
ExpectedOutputCleaner.cleanRoundingNoise(['Rejects (days)', 'Debugging (days)', 'Implementation (days)', 'Bandwidth',\
                                          'NEXT SPRINT (days)', 'NEXT SPRINT Bandwidth'],
                                        ['Committed'],
                                        test_greedyAllocation_EXPECTED,
                                        test_greedyAllocation_ACTUAL)

ExpectedOutputCleaner.cleanRoundingNoise(['Original Estimate', 'Effort Spent', 'Effort Remaining', \
                                          'Percent Achieved', 'Global Estimate', '% Global Done'],
                                        ['Tasks'],
                                        test_greedyAllocation_EXPECTED,
                                        test_greedyAllocation_ACTUAL)

tu_.EXPECTED['simm.test_greedyAllocation.Committed']      = test_greedyAllocation_EXPECTED['Committed']
tu_.EXPECTED['simm.test_greedyAllocation.Tasks']          = test_greedyAllocation_EXPECTED['Tasks']

tu_.ACTUAL['simm.test_greedyAllocation.Committed']        = test_greedyAllocation_ACTUAL['Committed']
tu_.ACTUAL['simm.test_greedyAllocation.Tasks']            = test_greedyAllocation_ACTUAL['Tasks']

tu_.testOK('simm.test_greedyAllocation.Committed'), \
tu_.testOK('simm.test_greedyAllocation.Tasks'), \

(True, True)

In [ ]:
test_greedyAllocation_ACTUAL['Committed'][:5]

In [ ]:
test_greedyAllocation_EXPECTED['Committed'][:5]

In [10]:
test_greedyAllocation_ACTUAL['Tasks'][:5]

,Owner,Task Type,Task Description,User Story Id,Planned for Sprint,Most Recently Assigned in Sprint,Delivered in Sprint,Original Estimate,Bucket,Effort Spent,Effort Remaining,Percent Achieved,Global Estimate,% Global Done
0,Anton Easterday,UNFINISHED_STORIES,Story implementation,UserStory #151,1,1,NOT_SET,2.0,CURRENT_SPRINT,0.0,2.0,0.0,2,0.0
1,Anton Easterday,UNFINISHED_STORIES,Story implementation,UserStory #97,1,1,NOT_SET,2.0,CURRENT_SPRINT,0.0,2.0,0.0,2,0.0
2,Anton Easterday,UNFINISHED_STORIES,Story implementation,UserStory #58,1,1,NOT_SET,4.0,CURRENT_SPRINT,0.0,4.0,0.0,4,0.0
3,Anton Easterday,UNFINISHED_STORIES,Story implementation,UserStory #83,1,1,NOT_SET,2.0,CURRENT_SPRINT,0.0,2.0,0.0,2,0.0
4,Beau Hockensmith,UNFINISHED_STORIES,Story implementation,UserStory #1,1,1,NOT_SET,10.0,CURRENT_SPRINT,0.0,10.0,0.0,10,0.0


In [ ]:
test_greedyAllocation_EXPECTED['Tasks'][:5]

<h1>test_balancedAllocation</h1>

In [18]:
# Implement test logic, and run it

#Test logic
def test_balancedAllocation():    
    output = {}
    RELEASE_DURATION = 125
    SPRINT_DURATION = 10

    # Configure models
    model = BalancedAllocationModel() 
    modelsConfig = ModelsConfig([], [], model)
    modelsConfig.random.reset(271)

    teams_df, stories_df, globalRepo = tu_.initTestData(tu_.DEV_DF, tu_.PM_DF, \
                                                        RELEASE_DURATION, SPRINT_DURATION, modelsConfig)

    modelsConfig.globalRepo = globalRepo
    # Select a team
    teamId = teams_df['Scrum Team'][0].teamId
    
    # Choose what to work on at the start of a sprint.
    SPRINT_DURATION = 10
    SPRINT = 1
    modelsConfig.context = ReleaseCycleContext(teamId, SPRINT, SPRINT_DURATION)
   
    work = WorkAssignments(modelsConfig.context, globalRepo)

    work = model.allocate(work, modelsConfig)
    
    committed_df = work.committedTime(SPRINT_DURATION)
    tasks_df = work.committedTasks()
        
    output['Committed'] = committed_df
    output['Tasks'] = tasks_df
    return output, work

# Run the test
test_balancedAllocation_ACTUAL, work = test_balancedAllocation()

In [19]:
# Uncomment to update expected output to match the actual one

# Helper method
def create_balancedAllocation_EXPECTED():
    tu_.createExpectedOutput(test_balancedAllocation_ACTUAL['Committed'],    'simm.test_balancedAllocation.Committed')
    tu_.createExpectedOutput(test_balancedAllocation_ACTUAL['Tasks'],        'simm.test_balancedAllocation.Tasks')

# Uncomment to update expected output to match the actual one, and then put the comment back
#create_balancedAllocation_EXPECTED()

In [23]:
# Load expected output, update the EXPECTED and ACTUAL dictionaries, and check test is OK
test_balancedAllocation_EXPECTED = {}

test_balancedAllocation_EXPECTED['Committed']      = tu_.loadExpectedOutput('simm.test_balancedAllocation.Committed')
test_balancedAllocation_EXPECTED['Tasks']          = tu_.loadExpectedOutput('simm.test_balancedAllocation.Tasks')

# Rounding inaccuracies in saving and loading CSV will create an artificial mismatch between ACTUAL and EXPECTED
# So round EXPECTED and ACTUAL to 6 decimal places for sensitive fields (any float)
ExpectedOutputCleaner.cleanRoundingNoise(['Rejects (days)', 'Debugging (days)', 'Implementation (days)', 'Bandwidth',\
                                          'NEXT SPRINT (days)', 'NEXT SPRINT Bandwidth'],
                                        ['Committed'],
                                        test_balancedAllocation_EXPECTED,
                                        test_balancedAllocation_ACTUAL)
ExpectedOutputCleaner.cleanRoundingNoise(['Original Estimate', 'Effort Spent', 'Effort Remaining', \
                                          'Percent Achieved', 'Global Estimate', '% Global Done'],
                                        ['Tasks'],
                                        test_balancedAllocation_EXPECTED,
                                        test_balancedAllocation_ACTUAL)

tu_.EXPECTED['simm.test_balancedAllocation.Committed']      = test_balancedAllocation_EXPECTED['Committed']
tu_.EXPECTED['simm.test_balancedAllocation.Tasks']          = test_balancedAllocation_EXPECTED['Tasks']

tu_.ACTUAL['simm.test_balancedAllocation.Committed']        = test_balancedAllocation_ACTUAL['Committed']
tu_.ACTUAL['simm.test_balancedAllocation.Tasks']            = test_balancedAllocation_ACTUAL['Tasks']

tu_.testOK('simm.test_balancedAllocation.Committed'), \
tu_.testOK('simm.test_balancedAllocation.Tasks'), \

(True, True)

In [ ]:
test_balancedAllocation_ACTUAL['Committed'][:5]

In [ ]:
test_balancedAllocation_EXPECTED['Committed'][:5]

In [ ]:
test_balancedAllocation_ACTUAL['Tasks'][:5]

In [ ]:
test_balancedAllocation_EXPECTED['Tasks'][:5]

<h1>test_noLaggardsAllocation</h1>

In [29]:
# Implement test logic, and run it

#Test logic
def test_noLaggardsAllocation():    
    output = {}
    RELEASE_DURATION = 125
    SPRINT_DURATION = 10

    # Configure models
    model = NoLaggardsAllocationModel() 
    modelsConfig = ModelsConfig([], [], model)
    modelsConfig.random.reset(271)

    teams_df, stories_df, globalRepo = tu_.initTestData(tu_.DEV_DF, tu_.PM_DF, \
                                                        RELEASE_DURATION, SPRINT_DURATION, modelsConfig)

    modelsConfig.globalRepo = globalRepo
    # Select a team
    teamId = teams_df['Scrum Team'][0].teamId
    
    # Choose what to work on at the start of a sprint.
    SPRINT_DURATION = 10
    SPRINT = 1
    modelsConfig.context = ReleaseCycleContext(teamId, SPRINT, SPRINT_DURATION)
   
    work = WorkAssignments(modelsConfig.context, globalRepo)

    work = model.allocate(work, modelsConfig)
    
    committed_df = work.committedTime(SPRINT_DURATION)
    tasks_df = work.committedTasks()
        
    output['Committed'] = committed_df
    output['Tasks'] = tasks_df
    return output, work

# Run the test
test_noLaggardsAllocation_ACTUAL, work = test_noLaggardsAllocation()

In [30]:
# Uncomment to update expected output to match the actual one

# Helper method
def create_noLaggardsAllocation_EXPECTED():
    tu_.createExpectedOutput(test_noLaggardsAllocation_ACTUAL['Committed'],    'simm.test_noLaggardsAllocation.Committed')
    tu_.createExpectedOutput(test_noLaggardsAllocation_ACTUAL['Tasks'],        'simm.test_noLaggardsAllocation.Tasks')

# Uncomment to update expected output to match the actual one, and then put the comment back
#create_noLaggardsAllocation_EXPECTED()

In [31]:
# Load expected output, update the EXPECTED and ACTUAL dictionaries, and check test is OK
test_noLaggardsAllocation_EXPECTED = {}

test_noLaggardsAllocation_EXPECTED['Committed']      = tu_.loadExpectedOutput('simm.test_noLaggardsAllocation.Committed')
test_noLaggardsAllocation_EXPECTED['Tasks']          = tu_.loadExpectedOutput('simm.test_noLaggardsAllocation.Tasks')

# Rounding inaccuracies in saving and loading CSV will create an artificial mismatch between ACTUAL and EXPECTED
# So round EXPECTED and ACTUAL to 6 decimal places for sensitive fields (any float)
ExpectedOutputCleaner.cleanRoundingNoise(['Rejects (days)', 'Debugging (days)', 'Implementation (days)', 'Bandwidth',\
                                          'NEXT SPRINT (days)', 'NEXT SPRINT Bandwidth'],
                                        ['Committed'],
                                        test_noLaggardsAllocation_EXPECTED,
                                        test_noLaggardsAllocation_ACTUAL)
ExpectedOutputCleaner.cleanRoundingNoise(['Original Estimate', 'Effort Spent', 'Effort Remaining', \
                                          'Percent Achieved', 'Global Estimate', '% Global Done'],
                                        ['Tasks'],
                                        test_noLaggardsAllocation_EXPECTED,
                                        test_noLaggardsAllocation_ACTUAL)

tu_.EXPECTED['simm.test_noLaggardsAllocation.Committed']      = test_noLaggardsAllocation_EXPECTED['Committed']
tu_.EXPECTED['simm.test_noLaggardsAllocation.Tasks']          = test_noLaggardsAllocation_EXPECTED['Tasks']

tu_.ACTUAL['simm.test_noLaggardsAllocation.Committed']        = test_noLaggardsAllocation_ACTUAL['Committed']
tu_.ACTUAL['simm.test_noLaggardsAllocation.Tasks']            = test_noLaggardsAllocation_ACTUAL['Tasks']

tu_.testOK('simm.test_noLaggardsAllocation.Committed'), \
tu_.testOK('simm.test_noLaggardsAllocation.Tasks'), \

(True, True)

In [ ]:
test_noLaggardsAllocation_ACTUAL['Committed'][:5]

In [ ]:
test_noLaggardsAllocation_EXPECTED['Committed'][:5]

In [ ]:
test_noLaggardsAllocation_ACTUAL['Tasks'][:5]

In [ ]:
test_noLaggardsAllocation_EXPECTED['Tasks'][:5]

<h1>test_distributedLagQualityModel</h1>

In [32]:
# Implement test logic, and run it

#Test logic
def test_distributedLagQualityModel():
    output = {}
    RELEASE_DURATION = 125
    SPRINT_DURATION = 10

    # Configure models
    modelsConfig = ModelsConfig([DefaultCostModel()], [DistributedLagQualityModel()], GreedyAllocationModel()) 
    modelsConfig.random.reset(271)

    teams_df, stories_df, globalRepo = tu_.initTestData(tu_.DEV_DF, tu_.PM_DF, \
                                                        RELEASE_DURATION, SPRINT_DURATION, modelsConfig)
    modelsConfig.globalRepo = globalRepo
    
    TEAM_ID = 'Team A'
    SPRINT = 1
    ctx = ReleaseCycleContext(TEAM_ID, SPRINT, SPRINT_DURATION)
    modelsConfig.context = ctx
    
    # Pretend all stories have been completed on the first sprint
    uss_list = []
    ids = globalRepo.storiesRepo.findIds()
    for userStoryId in ids:
        uss = globalRepo.teamsRepo.getUserStoryStatus(userStoryId)
        uss.percentAchieved = 1.0
        uss.planned = True
        uss.sprintPlanned = 1
        uss.sprintDelivered = 1
    
    # Now pretend we raverse the next 5 sprints, generating bugs
    bugs = []
    for i in range(5):
        ctx.sprint = SPRINT + i + 1
        qualityModel = modelsConfig.qualityModels[0]
        bugs.extend(qualityModel.findBugs(modelsConfig))
    
    bugs_df = Ticket.build_bugs_df(bugs)
    
    stories_df = UserStory.build_stories_df(modelsConfig.globalRepo)
    
    output['bugs'] = bugs_df
    output['stories'] = stories_df

    return output

# Run the test
test_distributedLagQualityModel_ACTUAL = test_distributedLagQualityModel()

In [33]:
# Uncomment to update expected output to match the actual one

# Helper method
def create_distributedLagQualityModel_EXPECTED():
    tu_.createExpectedOutput(test_distributedLagQualityModel_ACTUAL['bugs'],    'simm.test_distributedLagQualityModel.bugs')
    tu_.createExpectedOutput(test_distributedLagQualityModel_ACTUAL['stories'], 'simm.test_distributedLagQualityModel.stories')

# Uncomment to update expected output to match the actual one, and then put the comment back
#create_distributedLagQualityModel_EXPECTED()

In [34]:
# Load expected output, update the EXPECTED and ACTUAL dictionaries, and check test is OK
list_cols_bugs = [] # Lists are loaded as strings, so require special processing on load
list_cols_stories = ['Open Bugs', 'Closed Bugs']
test_distributedLagQualityModel_EXPECTED = {}

test_distributedLagQualityModel_EXPECTED['bugs']     = tu_.loadExpectedOutput('simm.test_distributedLagQualityModel.bugs', 
                                                                              list_cols_bugs)
test_distributedLagQualityModel_EXPECTED['stories']  = tu_.loadExpectedOutput('simm.test_distributedLagQualityModel.stories', 
                                                                              list_cols_stories)

# Rounding inaccuracies in saving and loading CSV will create an artificial mismatch between ACTUAL and EXPECTED
# So round EXPECTED and ACTUAL to 6 decimal places for sensitive fields (any float)

ExpectedOutputCleaner.cleanRoundingNoise(['Estimated Cost', 'Effort to Date', 'Percent Achieved'],
                                        ['bugs'],
                                        test_distributedLagQualityModel_EXPECTED,
                                        test_distributedLagQualityModel_ACTUAL)

tu_.EXPECTED['simm.test_distributedLagQualityModel.bugs']        = test_distributedLagQualityModel_EXPECTED['bugs']
tu_.EXPECTED['simm.test_distributedLagQualityModel.stories']     = test_distributedLagQualityModel_EXPECTED['stories']

tu_.ACTUAL['simm.test_distributedLagQualityModel.bugs']          = test_distributedLagQualityModel_ACTUAL['bugs']
tu_.ACTUAL['simm.test_distributedLagQualityModel.stories']       = test_distributedLagQualityModel_ACTUAL['stories']

tu_.testOK('simm.test_distributedLagQualityModel.bugs'), \
tu_.testOK('simm.test_distributedLagQualityModel.stories'), \

(True, True)

In [ ]:
test_distributedLagQualityModel_ACTUAL['bugs'][:8]

In [ ]:
test_distributedLagQualityModel_EXPECTED['bugs'][:8]

In [ ]:
test_distributedLagQualityModel_ACTUAL['stories'][:8]

In [ ]:
test_distributedLagQualityModel_EXPECTED['stories'][:8]